In [1]:
#@title
!pip install git+https://www.github.com/keras-team/keras-contrib.git
!pip install pillow

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-az9c1aha
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-az9c1aha
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=af4044fd7aaaa6abdb94774abcae55c0f13d4e04d30b4be8e56d199eb9cc5c3d
  Stored in directory: /tmp/pip-ephem-wheel-cache-sdupa4rc/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [2]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

In [3]:
import imageio
from skimage.transform import resize
from PIL import Image

In [4]:
from google.colab import drive 
drive.mount('/content/drive/')
root_dir = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
class DataLoader():
    def __init__(self, dataset_name, data_type, img_res=(64, 64)): #128, 128
        self.dataset_name = dataset_name
        self.data_type = data_type
        self.img_res = img_res
        
    def load_data(self, batch_size=1, is_testing=False):
        path = glob('/content/drive/My Drive/Colab Notebooks/datasets/%s/%s/*' % (self.dataset_name, self.data_type))
        batch_images = np.random.choice(path, size=batch_size)
        imgs_A = []
        imgs_B = []
        for img_path in batch_images:
            img = self.imread(img_path)
            img_np = np.array(img)/255
            h, w, _ = img_np.shape
            _w = int(w/2)
            img_A, img_B = img_np[:, :_w, :], img_np[:, _w:, :]

            img_A = img_A * 255
            img_A = img_A.astype(np.uint8)
            img_A = Image.fromarray(img_A)

            img_B = img_B * 255
            img_B = img_B.astype(np.uint8)
            img_B = Image.fromarray(img_B)

            img_A = img_A.resize((self.img_res))
            img_B = img_B.resize((self.img_res))

            img_A = np.array(img_A)*255
            img_B = np.array(img_B)*255

            # If training => do random flip
            if not is_testing and np.random.random() < 0.5:
                img_A = np.fliplr(img_A)
                img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = (np.array(imgs_A)*255)/127.5 - 1.
        imgs_B = (np.array(imgs_B)*255)/127.5 - 1.

        return imgs_B, imgs_A

    def load_batch(self, batch_size=1, is_testing=False):
        path = glob('/content/drive/My Drive/Colab Notebooks/datasets/%s/%s/*' % (self.dataset_name, self.data_type))
        self.n_batches = int(len(path) / batch_size)
        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                img_np = np.array(img)/255;

                h, w, _ = img_np.shape
                half_w = int(w/2)
                img_A = img_np[:, :half_w, :]
                img_B = img_np[:, half_w:, :]

                img_A = img_A * 255
                img_A = img_A.astype(np.uint8)
                img_A = Image.fromarray(img_A)

                img_B = img_B * 255
                img_B = img_B.astype(np.uint8)
                img_B = Image.fromarray(img_B)

                img_A = img_A.resize((self.img_res))
                img_B = img_B.resize((self.img_res))

                img_A = np.array(img_A)*255
                img_B = np.array(img_B)*255
            
                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = (np.array(imgs_A)*255)/127.5 - 1.
            imgs_B = (np.array(imgs_B)*255)/127.5 - 1.

            yield imgs_B, imgs_A

    def imread(self, path):
        im = Image.open(path)
        return im.convert('RGB')

In [6]:
path_to_g_model = "/content/drive/My Drive/Colab Notebooks/datasets/turn_mnist/models/model_generator.h5"

In [7]:
from keras.models import load_model
import keras_contrib

In [8]:
g_model = load_model(path_to_g_model, custom_objects = {'InstanceNormalization': keras_contrib.layers.normalization.instancenormalization.InstanceNormalization}, compile=False)

In [9]:
dataset_name = 'turn_mnist_gen'
data_type_1 = "res_0_2"
data_type_2 = "res_1_3"
img_rows = 128
img_cols = 128
channels = 3

# Генерируем ложные "2" из реальных "0"
data_loader = DataLoader(dataset_name=dataset_name, data_type = data_type_1,
                                      img_res=(img_rows, img_cols) )

for i in range(0,1000):
    _, img_B = data_loader.load_data(batch_size=1, is_testing=True)
    generated_A = g_model.predict(img_B)
    generated_A_img = np.reshape(generated_A, (img_rows, img_cols, channels))  
    img_to_save = generated_A_img * 255
    img_to_save = img_to_save.astype(np.uint8)
    img_to_save = Image.fromarray(img_to_save)
    img_to_save.save( "/content/drive/My Drive/Colab Notebooks/datasets/turn_mnist_gen/gen_2/2_%d.jpg" % (i))

# Генерируем ложные "3" из реальных "1"
data_loader = DataLoader(dataset_name=dataset_name, data_type = data_type_2,
                                      img_res=(img_rows, img_cols))

for i in range(0,1000):
    _, img_B = data_loader.load_data(batch_size=1, is_testing=True)
    generated_A = g_model.predict(img_B)
    generated_A_img = np.reshape(generated_A, (img_rows, img_cols, channels))     
    img_to_save = generated_A_img * 255
    img_to_save = img_to_save.astype(np.uint8)
    img_to_save = Image.fromarray(img_to_save)
    img_to_save.save( "/content/drive/My Drive/Colab Notebooks/datasets/turn_mnist_gen/gen_3/3_%d.jpg" % (i))    

In [10]:
print("END_OF_PROGRAMM")

END_OF_PROGRAMM
